# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random as rd
# Visualizations will be shown in the notebook.
%matplotlib inline
from sklearn.utils import shuffle
import tensorflow as tf
import os
import cv2

In [ ]:


# TODO: Fill this in based on where you saved the training and testing data

training_file = 'data/train.p'
validation_file = 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:


# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(np.concatenate((y_train, y_valid, y_test),axis = None)))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
index = rd.randint(0, len(X_train)-1)
image = X_train[index]
print(y_train[index])
plt.imshow(image)
plt.imsave('./graphs/training_image.png', image)

In [ ]:
fig, ax = plt.subplots()
ax.hist(y_train, n_classes)
fig.savefig('./graphs/training_set_distributuion.png')

## Conclusion on data analysis

As it could be seen in the plot above the data set is not very balanced, which could cause a overfitting of the model to the classes which are over-represented in the dataset. To avoid this I will try to compendsate this by overfitting the dataset.

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE()
data_to_smote = X_train.reshape(n_train,32*32*3)
X_sm, y_sm = smote.fit_sample(data_to_smote, y_train)


In [ ]:
new_len = X_sm.shape[0]
X_sm = X_sm.reshape(new_len, 32, 32, 3)

In [ ]:
fig, ax = plt.subplots()
ax.hist(y_sm, n_classes)
fig.savefig('./graphs/training_set_distributuion_after_oversampling.png')

In [ ]:
def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    return tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

In [ ]:
def plot_images(dataset, n_images, samples_per_image):
    output = np.zeros((32 * n_images, 32 * samples_per_image, 3))
    
    row = 0
    for images in list(dataset.take(n_images).as_numpy_iterator()):
        print(images)
        output[:, row*32:(row+1)*32] = np.vstack(images.numpy())
        row += 1

    plt.figure()
    plt.imshow(output)
    plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
def normalize_image(image_set):
    normalized = (image_set - 128)/255
    normalized[:,:,:,0] = normalized[:,:,:,0] - np.mean(normalized[:,:,:,0])
    normalized[:,:,:,1] = normalized[:,:,:,1] - np.mean(normalized[:,:,:,1])
    normalized[:,:,:,2] = normalized[:,:,:,2] - np.mean(normalized[:,:,:,2])
    return normalized.astype('float32')

In [ ]:
print('Original mean over all images: ', np.mean(X_train))
print('Original min of all images: ', np.min(X_train))
print('Original max of all images: ', np.max(X_train))
X_train_norm = normalize_image(X_sm)
X_valid_norm = normalize_image(X_valid)
X_test_norm = normalize_image(X_test)
print('Normalized mean over all images: ', np.mean(X_train_norm))
print('Normalized min of all images: ', np.min(X_train_norm))
print('Normalized max of all images: ', np.max(X_train_norm))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(X_train_norm)
data = (X_sm[0:8] / 255).astype(np.float32)
exampleset = tf.data.Dataset.from_tensor_slices(data)
# Add augmentations
augmentations = [flip, color, zoom, rotate]

for f in augmentations:
    # Apply an augmentation only in 25% of the cases.
    examples = exampleset.map(f, num_parallel_calls=4)
    test_dataset = dataset.map(lambda x: tf.cond(tf.random_uniform([], 0, 1) > 0.75, lambda: f(x), lambda: x), num_parallel_calls=4)

In [ ]:
X_train_norm.shape

In [ ]:
examples = examples.map(lambda x: tf.clip_by_value(x, 0, 1))
plot_images(examples, n_images=10, samples_per_image=15)

In [ ]:
X_train_norm, y_train = shuffle(X_train_norm, y_train)

### Model Architecture

In [ ]:
EPOCHS = 20
BATCH_SIZE = 128
mu = 0
sigma = 0.1

In [ ]:


weights = {
    'layer_1': tf.Variable(tf.random_normal([3, 3, 3, 120], mean = mu, stddev= sigma), name='layer_1'),
    'layer_2': tf.Variable(tf.random_normal([3, 3, 120, 360], mean = mu, stddev= sigma), name='layer_2'),
    #'layer_3': tf.Variable(tf.random_normal([3, 3, 360, 120], mean = mu, stddev= sigma)),
    'layer_4': tf.Variable(tf.random_normal([6*6*360, 1200], mean = mu, stddev= sigma), name='layer_4'),
    'layer_5': tf.Variable(tf.random_normal([1200, 258], mean = mu, stddev= sigma), name='layer_5'),  
    'out': tf.Variable(tf.random_normal([258, n_classes], mean = mu, stddev= sigma), name='out')
    }
biases = {
    'layer_1': tf.Variable(tf.zeros([120]), name='layer_1'),
    'layer_2': tf.Variable(tf.zeros([360]), name='layer_2'),
    #'layer_3': tf.Variable(tf.zeros([120])),
    'layer_4': tf.Variable(tf.zeros([1200]), name='layer_4'),
    'layer_5': tf.Variable(tf.zeros([258]), name='layer_5'),
    'out': tf.Variable(tf.zeros([n_classes]), name='out')
    }

In [ ]:
def LeNet(x, keep_prop = 0.5):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer


    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x90.
    layer_1 = tf.nn.conv2d(x, weights['layer_1'], strides=[1, 1, 1, 1], padding='VALID')
    # Layer 1: Add bias
    layer_1 = tf.nn.bias_add(layer_1, biases['layer_1'])
    # Activation.
    layer_1 = tf.nn.relu(layer_1)
    # Dropout
    layer_1 = tf.nn.dropout(layer_1, keep_prop)
    # Pooling. Input = 28x28x90. Output = 14x14x90.
    layer_1 = tf.nn.max_pool(layer_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Layer 2: Convolutional. Output = 10x10x360.
    layer_2 = tf.nn.conv2d(layer_1, weights['layer_2'], strides=[1, 1, 1, 1], padding='VALID')
    # Layer 1: Add bias
    layer_2 = tf.nn.bias_add(layer_2, biases['layer_2'])
    # Activation.
    layer_2 = tf.nn.relu(layer_2)
    # Dropout
    layer_2 = tf.nn.dropout(layer_2, keep_prop)
    # Pooling. Input = 10x10x360. Output = 10x10x360.
    layer_2 = tf.nn.max_pool(layer_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    # Layer 3: Convolutional. Output = 6x6x120.
    #layer_3 = tf.nn.conv2d(layer_2, weights['layer_3'], strides=[1, 1, 1, 1], padding='VALID')
    # Layer 3: Add bias
    #layer_3 = tf.nn.bias_add(layer_3, biases['layer_3'])
    # Activation.
    #layer_3 = tf.nn.relu(layer_3)
    # Dropout
    #layer_3 = tf.nn.dropout(layer_3, keep_prop)
    # Pooling. Input = 6x6x360. Output = 3x3x120.
    #layer_3 = tf.nn.max_pool(layer_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Flatten. Input = 3x3x120. Output = 1080.
    layer_3 = tf.contrib.layers.flatten(layer_2)
    # Layer 3: Fully Connected. Input = 1080. Output = 540.
    layer_4 = tf.add(tf.matmul(layer_3, weights['layer_4']), biases['layer_4'])
    # Activation.
    layer_4 = tf.nn.relu(layer_4)
    layer_4 = tf.nn.dropout(layer_4, keep_prop)
    # Layer 4: Fully Connected. Input = 540. Output = 258.
    layer_5 = tf.add(tf.matmul(layer_4, weights['layer_5']), biases['layer_5'])
    # Activation.
    layer_5 = tf.nn.relu(layer_5)
    layer_5 = tf.nn.dropout(layer_5, keep_prop)
    # Layer 5: Fully Connected. Input = 258. Output = 43.
    logits = tf.add(tf.matmul(layer_5, weights['out']), biases['out'])
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3), name = "x")
y = tf.placeholder(tf.int32, (None), name = "y")
keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
rate = 0.001

logits = LeNet(x, keep_prob)
logits_named = tf.identity(logits, "logits")
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_operation_named = tf.identity(accuracy_operation, "accuracy_operation")

saver = tf.train.Saver()

In [ ]:
def evaluate(X_data, y_data, f_dropout):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: f_dropout})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train_norm)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train_norm, y_sm = shuffle(X_train_norm, y_sm)
        train_accuracy = 0
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_norm[offset:end], y_sm[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
            accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
            train_accuracy += (accuracy * len(batch_x))
        train_accuracy = train_accuracy / num_examples
        validation_accuracy = evaluate(X_valid_norm, y_valid, f_dropout = 1.0)
        print("EPOCH {} ...".format(i+1))
        print("Traning Accuracy = {:.3f}".format(train_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

In [ ]:
BATCH_SIZE = 128
with tf.Session() as sess:
    graph = tf.get_default_graph()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('lenet.meta')
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    
    print("Testing...")
    print()
    num_examples = len(X_test_norm)
    test_accuracy = 0
    #sess.tf.get_default_graph()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_test_norm[offset:offset+BATCH_SIZE], y_test[offset:offset+BATCH_SIZE]
        accuracy = sess.run("accuracy_operation:0", feed_dict={'x:0': batch_x, 'y:0': batch_y, 'keep_prob:0': 1.0})
        test_accuracy += (accuracy * len(batch_x))
    test_accuracy = test_accuracy / num_examples
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    print()

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
included_extensions = ['jpg','jpeg', 'bmp', 'png', 'gif']
test_image_paths = [fn for fn in os.listdir('./examples') if any(fn.endswith(ext) for ext in included_extensions)]
print(test_image_paths)
test_images = []
    
fig, ax = plt.subplots(2,len(test_image_paths),figsize=(20,10))
for i,path in enumerate(test_image_paths):
    print(i)
    image = plt.imread('./examples/'+path)
    
    if image.dtype == 'float32':
        image = np.uint8(image*255)
    print(np.max(image))
    print('Original Dimensions : ',image.shape)
    print('Original Max value : ', np.max(image))
    ax[0][i].imshow(image)
    resized = cv2.resize(image, (32, 32), interpolation = cv2.INTER_AREA)[:,:,0:3]
    ax[1][i].imshow(resized)

    test_images.append(resized)
test_images = normalize_image(np.array(test_images))
for test_image in test_images:
    print('Resized Dimensions : ',test_image.shape)
    print('Resized Max value : ', np.max(test_image))

### Predict the Sign Type for Each Image

In [ ]:
# Launch the graph
with tf.Session() as sess:
    graph = tf.get_default_graph()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('lenet.meta')
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    
    print("Testing...")
    print()
    output = sess.run("logits:0", feed_dict = {'x:0':test_images, 'keep_prob:0': 1.0})
    prop = sess.run(tf.nn.softmax(output))


In [ ]:
import csv
signnames = {}
with open('signnames.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        signnames[row['ClassId']] = row['SignName']
        
    print(signnames)

In [ ]:
for result in prop:
    print(len(result))
    print(np.argmax(result))

predicted = [signnames[str(np.argmax(result))] for result in output]
predicted

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
Correct = [23, 12, 11, 9, 14]
Correct_signs = [signnames[str(item)] for item in Correct]
print(Correct_signs)

In [ ]:
performance = 0
for pred, correct in zip(predicted, Correct_signs):
    performance += float(pred == correct)
performance /= len(Correct_signs)
print("Performance on internet images is {:.3f}%".format(100*performance))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
#print(output)
top_5 = []
with tf.Session() as sess:
    top_5 = sess.run(tf.nn.top_k(tf.constant(prop), k=5))
print("Top 5 probabilities are: ", top_5)

In [ ]:
print(len(top_5[0]))

fig, ax = plt.subplots(len(top_5[0]), 1,figsize=(20,30))
for i in range(len(top_5[0])):
    x_val = [signnames[str(index)] for index in top_5[1][i]]
    ax[i].barh(x_val, top_5[0][i])
    ax[i].set_xlim(0,2.0)
    ax[i].set_title(Correct_signs[i])
    ax[i].set_xlabel('Probability')
fig.savefig('./graphs/top_5_example_images.png')    

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
layer_1 = tf.Variable(tf.Variable(tf.random_normal([3, 3, 3, 120]), name='layer_1'))

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
outputFeatureMap(test_images[1], layer_1)